In [ ]:
import os,glob
import re
import pandas as pd
import numpy as np
from datetime import date

pr_ref_header_list =['Report Date', 'Country', 'Product Barcode', 'Flipkart ProductID', 'Flipkart ProductName', 'Category Level1', 'Category Level2',
       'Category Level3', 'Category Level4', 'Site', 'Number Of offers', 'Sku Id', 'Brand', 'Product Name', 'Size', 'Color', 'Other Variants',
       'Currency', 'Was_Price', 'Now Price', 'Discount', 'Shipping Cost', 'Product Availability', 'Stock Information', 'Seller Name',
       'Buybox Flag', 'Seller Type', 'Min Price Flag', 'Limited_Stock_Info', 'Warranty', 'Warranty Period', 'Product Condition', 'Product InputUrl',
       'Product Output Url', 'Timestamp', 'Extract Date', 'Category from site1', 'Category from site2', 'Category from site3',
       'Category from site4', 'Category from site5', 'Category from site6', 'Fulfillment', 'Extra2', 'Extra3', 'Extra4']

# Current Working Directory
lookup_dir = os.getcwd()
# Client mapping file df - Hardcoded path
client_map_file_path = "E:\Flipkart_Price_Refresh\Input_lookup_file\price_v2_template_data_updated_SQL - All.xlsx"
df_client_map = pd.read_excel(client_map_file_path)
# Input csv files path
input_csv_lst = glob.glob(os.path.join(lookup_dir+'\\Input\\','*.csv'))

def column_validation():
    _ , final_column_count = df_final_output.shape
    
    if pr_ref_header_list != list(df_final_output.columns):
        if final_column_count == 46:
            return "\033[1m" + "\033[91m" + "  |  " + "Warning: " + "Headers are different"
        else:
            return "\033[1m" + "\033[91m" + "  |  " + "Warning: " + str(final_column_count) + " Columns"
    else:
        return ""

def row_validation():
    before_lookup_row_count, _ = df.shape
    final_row_count, _ = df_final_output.shape
    missing_rows = before_lookup_row_count - final_row_count
    
    if before_lookup_row_count != final_row_count:
        return "\033[1m" + "\033[91m" + "  |  " + "Warning: " + str(missing_rows) + " rows missing after lookup"
    else:
        return ""
    
def date_validation():    
    today = date.today()
    dt_string = today.strftime('%Y-%m-%d')
    
    report_date_validate = df_final_output['Report Date'].str.contains(dt_string).all(skipna=True)
    timestamp_validate = df_final_output['Timestamp'].str.contains(dt_string).all(skipna=True)
    extract_date_validate = df_final_output['Extract Date'].str.contains(dt_string).all(skipna=True)
    
    if report_date_validate == False or timestamp_validate == False or extract_date_validate == False:
        return "\033[1m" + "\033[91m" + "  |  " + "Warning: " + "Date is different"
    else:
        return ""

def comma_validation():
    # Todo
    if comma_replace:
        return "\033[1m" + "\033[91m" + "  |  " + "Warning: " + "Comma not replaced"
    else:
        return ""
    
# ----- Lookup Code -----

url_status_count_list = []

for file in input_csv_lst:
    # Input csv file df
    file_name_csv = file.split('\\')[-1].replace('.csv', '')
    file_path_csv = file
    df = pd.read_csv(file_path_csv)
    
    # Check and remove all NaN rows 
    df.dropna(axis=0, how='all', inplace=True)
    
    # merge, conversion, and deletion
    df_merged = df.merge(df_client_map, on='Extra3')
    
    # Todo: Del
    df_merged_test = df_merged.copy(deep=True)
    
    
    int_conv_col_lst_var = ['Number Of offers', 'Stock Information', 'Buybox Flag', 'Min Price Flag', 'Product Barcode', 'Flipkart ProductID']
    int_conv_col_lst_val = ['Number Of offers', 'Stock Information', 'Buybox Flag', 'Min Price Flag', 'item_barcode', ' Flipkart_Internal_ID ']
    float_conv_col_lst_var = ['Now Price']
    float_conv_col_lst_val = ['Now Price']
    rename_col_lst_var = ['Flipkart ProductName', 'Category Level1', 'Category Level2', 'Category Level3', 'Category Level4']
    rename_col_lst_val = [' Flipkart_product_online_name', 'Dept (Category_level_1)', 'Section (Category_level_2)', 'Family (Category_level_3)', 'Sub_Family(Category_level_4)']
    del_col_name_lst = [' Flipkart_product_online_name', 'Competitor', 'competitor_url', 'item_barcode', ' Flipkart_Internal_ID ', 'input_B', 'Dept (Category_level_1)', 'Section (Category_level_2)', 'Family (Category_level_3)', 'Sub_Family(Category_level_4)', 'Competitor_Internal_product ID']
    # 'Unnamed: 12' del -  removed from file
    
    # safely convert non-numeric type
    #df_merged[int_conv_col_lst_var] = df_merged[int_conv_col_lst_val].apply(pd.to_numeric, errors='raise')
    df_merged[int_conv_col_lst_var] = df_merged[int_conv_col_lst_val].apply(pd.to_numeric, errors='ignore')
    df_merged[float_conv_col_lst_var] = df_merged[float_conv_col_lst_val].apply(pd.to_numeric, errors='raise')
    
    df_merged[rename_col_lst_var] = df_merged[rename_col_lst_val]
    df_merged.drop(del_col_name_lst, axis=1, inplace=True)
    #del df_merged[del_col_name_lst]
    #df_merged['Product Barcode'] = df_merged['item_barcode'].apply(lambda p: barcode(p) if p == p else '')
    
    # Final Output Dataframe
    df_final_output = df_merged
    df_final_output.to_csv(lookup_dir + '\\Output\\' + file_name_csv + '.csv',index=False)
    
    # Url status count dict append and df conv
    url_status_columns = ['file_name_csv', 'unique_urls', 'total_listings','Valid URLs', 'Blocked URLs', 'Invalid URLs(inactive)', 'Valid URLs OOS (Without price)', 'Script time out URLs']
    url_status_count_dict = dict(df_final_output['Extra2'].value_counts())
    url_status_count_dict['file_name_csv'] = file_name_csv
    url_status_count_dict['unique_urls'] = df_final_output['Extra3'].nunique()
    url_status_count_dict['total_listings'] = df_final_output.shape[0]
    url_status_count_list.append(url_status_count_dict)
    df_url_status = pd.DataFrame(url_status_count_list, columns=url_status_columns)
    
    # Validation function call
    print(file_name_csv + str(df_final_output.shape) + column_validation() + row_validation() + date_validation(), end="\n\x1b[0m")
    
print("\n \033[1m" + "\033[92m" + "---------- Lookup Completed ----------")

# Summary data count
df_url_status.iloc[:,1:] = df_url_status.iloc[:,1:].astype('Int64').astype(object).fillna('-')
df_url_status.index+=1
df_url_status

### Old lookup code

In [ ]:
import pandas as pd
import numpy as np
import time
import os,glob
import math as ma
def barcode(x):
    if x == x:
        try:
            return int(x)
        except:return x
InputPath=os.getcwd()
InputCSV=glob.glob(os.path.join(InputPath+'//Input_lookup_file//','*.xlsx'))
for i in InputCSV:
    InputDF=pd.read_excel(i)

OutputPath=os.getcwd()
OutputCSV=glob.glob(os.path.join(OutputPath+'//Input//','*.csv'))
for f in OutputCSV:
    OutputDF=pd.read_csv(f)
    filename=f.split('\\')[-1].replace('.csv','')
    FinalPath=os.getcwd()
    file=OutputDF.merge(InputDF,on='Extra3')
    # Souq column conversion to int - Change column format 0  in SKU ID before saving cvs file
    # Uncomment Below line ['Sku Id'] for Souq Egypt
#     file['Sku Id']=file['Sku Id'].apply(lambda t: int(t) if t == t else '')
    file['Number Of offers']=file['Number Of offers'].apply(lambda a: int(a) if a == a else '')
    file['Now Price']=file['Now Price'].apply(lambda a: float(a) if a == a else '')
    file['Stock Information']=file['Stock Information'].apply(lambda b: int(b) if b == b else '')
    file['Buybox Flag']=file['Buybox Flag'].apply(lambda y: int(y) if y == y else '')
    file['Min Price Flag']=file['Min Price Flag'].apply(lambda z: int(z) if z == z else '')
    file['Flipkart ProductName']=file[' Flipkart_product_online_name']
#     Noon_uae - For barcode comment before .apply(lambda p: int(p) if p == p else '')
    file['Product Barcode']=file['item_barcode'].apply(lambda p: barcode(p) if p == p else '')
    file['Flipkart ProductID']=file[' Flipkart_Internal_ID '].apply(lambda x: barcode(x) if x == x else '')
    file['Category Level1']=file['Dept (Category_level_1)']
    file['Category Level2']=file['Section (Category_level_2)']
    file['Category Level3']=file['Family (Category_level_3)']
    file['Category Level4']=file['Sub_Family(Category_level_4)']
    #file['Category Level4']=file['Sub_Family(Category_level_4)']`a
    del file[' Flipkart_product_online_name']
    del file['Competitor']
    del file['competitor_url']
    del file['item_barcode']
    del file[' Flipkart_Internal_ID ']
    del file['input_B']
    del file['Dept (Category_level_1)']
    del file['Section (Category_level_2)']
    del file['Family (Category_level_3)']
    del file['Sub_Family(Category_level_4)']
    del file['Competitor_Internal_product ID']
    del file['Unnamed: 12']
#    del file['Unnamed: 13']      #uncomment for KSA and Egypt and comment for UAE
#    del file['Unnamed: 14']     #uncomment for KSA and comment for Egypt and UAE
#     del file['Unnamed: 47']     #uncomment for KSA and comment for Egypt and UAE
    finalOutput=file.to_csv(FinalPath+'\\Output\\'+filename+'.csv',index=False)
    